**Note to grader:** Each question consists of parts, e.g. Q1(i), Q1(ii), etc. Each part must be first graded  on a 0-4 scale, following the standard NJIT convention (A:4, B+: 3.5, B:3, C+: 2.5, C: 2, D:1, F:0). However, any given item may be worth 4 or 8 points; if an item is worth 8 points, you need to accordingly scale the 0-4 grade.


The total score must be re-scaled to 100. That should apply to all future assignments so that Canvas assigns the same weight on all assignments.



# Assignment 2



### Preparation Steps




We will work with this [mystery dataset](https://drive.google.com/open?id=1WLnWBThCYZ25pReI5DCwk2bgDaCrJxI_&authuser=ikoutis%40njit.edu&usp=drive_fs) that you can download and place to your google drive. You can then put it somewhere on your google drive and bring it into your Colab by following the steps in the following cell.



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import scipy.io

#mat = scipy.io.loadmat('/content/gdrive/Shareddrives/graph-modification/data/mysteryDataset.mat')

Mounted at /content/gdrive


The file contains

* Two matrices $X$ and $X_1$ of numerical features. These datasets have the same dimensions (169343x80) but they are different.
* An array $y$ of labels, ranging from 0-39.
* The indices $otrain$ of a training set. These indices tell you what rows of the arrays $X,X_1,y$ correspond to the training points. You can use these to make two different training sets $(X[train], y[train])$ and $(X_1[train], y[train])$
* Similarly, it contains the indexes for a validation and a test set, $ovalid$ and $otest$ respectively.

The following cell shows how to access these arrays and assign them to local numpy objects.

In [ ]:
mat = scipy.io.loadmat('mysteryDataset.mat')

In [ ]:
# this is an example
#ft = mat.get('X')

_________________________________________________

## <font color = 'blue'> Question 1. Import the dataset and conver to torch tensors </font>

Your task for this question is to adapt the above preparation steps, import all mentioned variables into numpy arrays, and then transform them to PyTorch tensors.


In [2]:
# Adapting the above preparation steps

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import scipy.io

# Downloading data
mat = scipy.io.loadmat('/content/gdrive/MyDrive/Colab Notebooks/CS677/mysteryDataset.mat')

Mounted at /content/gdrive


In [3]:
# Importing all mentioned variables into numpy arrays, and then transform them to PyTorch tensors

import torch

# Converting to tensors
for key in ['X','X1','y','otrain','ovalid','otest']:
  mat[key] = torch.from_numpy(mat[key])
  if key in ['X','X1','y']: mat[key] = mat[key].float()

In [4]:
# Assigning to local variables

X = mat['X']
X1 = mat['X1']
y = mat['y']
otrain = mat['otrain']
ovalid = mat['ovalid']
otest = mat['otest']
del mat # Delete the matrix to save memory

In [ ]:
# for grader use only

# insert grade here  (out of 4)

# G[1] =
#
# please justify point subtractions when needed

_________________________________________________

## <font color = 'blue'> Question 2. Write a functioning classifier in PyTorch </font>

Write code that defines a classification model for the above dataset, and all other functions that are needed for its training. Apply your model on the two datsets $X,X_1$ and report the accuracy. The classifier should operate on the GPU.

**Hint:** Re-use code we discussed for the Softmax Regression module.

In [5]:
from torch.utils.data import DataLoader

# Weights initializer
def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, std=0.01)

# Accuracy function
def accuracy(X,y):
  return (model(X.to(device))[0].argmax(axis=1) == y.to(device)[0].reshape(-1)).sum()/y.shape[1]

# Creating dataloader
def dataloader(X,y,batch_size):
  return DataLoader([[X[0][i],y[0][i]] for i in range(X.shape[1])],
                    batch_size = batch_size,
                    shuffle = True)

In [6]:
from torch import nn

# Model
class SoftMaxRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.LazyLinear(40)

  def forward(self, x):
    return self.linear(x)

In [7]:
# Creating train, validation, and test sets

X_train, X1_train, y_train = X[otrain], X1[otrain], y[otrain]
X_valid, X1_valid, y_valid = X[ovalid], X1[ovalid], y[ovalid]
X_test, X1_test, y_test = X[otest], X1[otest], y[otest]

_________________________________________________

In [8]:
# DATASET X

import torch.nn.functional as F

# Hyperparameters
batch_size = 1000
n_epochs = 500
lr = 10e-3

# Creating dataloader
torch.manual_seed(0)
train_iter = dataloader(X_train,y_train,batch_size)

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Assigning to GPU if available

model = SoftMaxRegression().to(device) # Model
loss_fn = nn.CrossEntropyLoss() # Loss function
optimizer = torch.optim.Adam(model.parameters(),lr=lr) # Optimizer

model.apply(init_weights) # Initialize Weights

for epoch in range(1,n_epochs+1):
  total_loss = 0
  for x_batch, y_batch in train_iter:

    # Geting batches
    x_batch = x_batch.to(device)
    y_batch = F.one_hot(y_batch.reshape(-1).long(),40).to(device).float()

    model.train() # Puting model into training mode
    yhat = model(x_batch) # Get prediction
    loss = loss_fn(yhat, y_batch) # Calculate loss

    # Updating weights
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss += loss/batch_size

  if not epoch % 50:
    model.eval()
    print(epoch,total_loss)
    print(accuracy(X_train,y_train),accuracy(X_valid,y_valid))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


50 tensor(0.2644, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1914, device='cuda:0') tensor(0.0788, device='cuda:0')
100 tensor(0.2532, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2104, device='cuda:0') tensor(0.0859, device='cuda:0')
150 tensor(0.2444, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2936, device='cuda:0') tensor(0.2610, device='cuda:0')
200 tensor(0.2372, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3353, device='cuda:0') tensor(0.3228, device='cuda:0')
250 tensor(0.2311, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3403, device='cuda:0') tensor(0.3251, device='cuda:0')
300 tensor(0.2259, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3490, device='cuda:0') tensor(0.3286, device='cuda:0')
350 tensor(0.2215, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3561, device='cuda:0') tensor(0.3326, device='cuda:0')
400 tensor(0.2177, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3652, device='cuda:0') tensor(0.3358, device='cuda:0')
450 tenso

In [9]:
# Evaluating the model

model.eval()
print("Train Accuracy:",accuracy(X_train,y_train))
print("Test Accuracy:",accuracy(X_test,y_test))

Train Accuracy: tensor(0.3802, device='cuda:0')
Test Accuracy: tensor(0.3032, device='cuda:0')


_________________________________________________

In [10]:
# DATASET X1

import torch.nn.functional as F

# Hyperparameters
batch_size = 1000
n_epochs = 500
lr = 10e-3

# Creating dataloader
torch.manual_seed(0)
train_iter = dataloader(X1_train,y_train,batch_size)

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Assigning to GPU if available

model = SoftMaxRegression().to(device) # Model
loss_fn = nn.CrossEntropyLoss() # Loss function
optimizer = torch.optim.Adam(model.parameters(),lr=lr) # Optimizer

model.apply(init_weights) # Initialize Weights

for epoch in range(1,n_epochs+1):
  total_loss = 0
  for x_batch, y_batch in train_iter:

    # Geting batches
    x_batch = x_batch.to(device)
    y_batch = F.one_hot(y_batch.reshape(-1).long(),40).to(device).float()

    model.train() # Puting model into training mode
    yhat = model(x_batch) # Get prediction
    loss = loss_fn(yhat, y_batch) # Calculate loss

    # Updating weights
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss += loss/batch_size

  if not epoch % 50:
    model.eval()
    print(epoch,total_loss)
    print(accuracy(X1_train,y_train),accuracy(X1_valid,y_valid))

50 tensor(0.1559, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5143, device='cuda:0') tensor(0.5140, device='cuda:0')
100 tensor(0.1557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5144, device='cuda:0') tensor(0.5168, device='cuda:0')
150 tensor(0.1557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5154, device='cuda:0') tensor(0.5159, device='cuda:0')
200 tensor(0.1556, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5150, device='cuda:0') tensor(0.5192, device='cuda:0')
250 tensor(0.1557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5154, device='cuda:0') tensor(0.5196, device='cuda:0')
300 tensor(0.1557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5156, device='cuda:0') tensor(0.5215, device='cuda:0')
350 tensor(0.1556, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5147, device='cuda:0') tensor(0.5175, device='cuda:0')
400 tensor(0.1556, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5160, device='cuda:0') tensor(0.5169, device='cuda:0')
450 tenso

In [11]:
# Evaluating the model

model.eval()
print("Train Accuracy:",accuracy(X1_train,y_train))
print("Test Accuracy:",accuracy(X1_test,y_test))

Train Accuracy: tensor(0.5164, device='cuda:0')
Test Accuracy: tensor(0.4969, device='cuda:0')


_________________________________________________

### Answer Q.2

Using a simple soft max classifier (lr=10e-3, epochs = 500, batch_size = 1000, no hidden layers), the accuracy was the following:

Dataset X:
- Train Accuracy: 38.02%
- Test Accuracy: 30.32%

Dataset X1:
- Train Accuracy: 51.64%
- Test Accuracy: 49.69%

In [ ]:
# for grader use only

# insert grade here  (out of 8)

# G[2] =
#
# please justify point subtractions when needed

_________________________________________________

## <font color = 'blue'> Question 3. Maximize the accuracy on the two datasets </font>

Augment your classifier from Question-2 with any number and type of layers you want, with the goal to maximize the **validation** accuracy you achieve on the two datasets. Feel free to use any stopping criterion you want for the training process. The networks for $X$ and $X_1$ do not have be of the same architecture.

Show your code, and add a text cell summarizing your idea and findings. Finally apply your models to the **test** set, and report the accuracy. Feel free to discuss your validation accuracy on Canvas. Also please avoid looking at the test set, until the very end.

**Rubric**: All complete answers get 8 points, and the **top 5** test accuracies reported get an extra 10\% in the final quiz.

In [12]:
# DATASET X

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.LazyLinear(1000),
        nn.ReLU(),
        nn.LazyLinear(500),
        nn.ReLU(),
        nn.LazyLinear(40)
    )

  def forward(self, x):
    return self.net(x)

In [13]:
import torch.nn.functional as F

# Hyperparameters
batch_size = 500
n_epochs = 2000
lr = 10e-3

# Creating dataloader
torch.manual_seed(0)
train_iter = dataloader(X_train,y_train,batch_size)

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Assigning to GPU if available

model = Net().to(device) # Model
loss_fn = nn.CrossEntropyLoss() # Loss function
optimizer = torch.optim.Adam(model.parameters(),lr=lr) # Optimizer

model.apply(init_weights) # Initialize Weights

for epoch in range(1,n_epochs+1):
  total_loss = 0
  for x_batch, y_batch in train_iter:

    # Geting batches
    x_batch = x_batch.to(device)
    y_batch = F.one_hot(y_batch.reshape(-1).long(),40).to(device).float()

    model.train() # Puting model into training mode
    yhat = model(x_batch) # Get prediction
    loss = loss_fn(yhat, y_batch) # Calculate loss

    # Updating weights
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss += loss/batch_size

  if not epoch % 50:
    model.eval()
    print(epoch,total_loss)
    print(accuracy(X_train,y_train),accuracy(X_valid,y_valid))

50 tensor(0.5338, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5850, device='cuda:0') tensor(0.5184, device='cuda:0')
100 tensor(0.4962, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5931, device='cuda:0') tensor(0.5174, device='cuda:0')
150 tensor(0.4764, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6243, device='cuda:0') tensor(0.5938, device='cuda:0')
200 tensor(0.4689, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6295, device='cuda:0') tensor(0.5939, device='cuda:0')
250 tensor(0.4596, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6273, device='cuda:0') tensor(0.5763, device='cuda:0')
300 tensor(0.4565, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6429, device='cuda:0') tensor(0.6183, device='cuda:0')
350 tensor(0.4514, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6524, device='cuda:0') tensor(0.6189, device='cuda:0')
400 tensor(0.4485, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6514, device='cuda:0') tensor(0.6128, device='cuda:0')
450 tenso

In [14]:
# Evaluating the model

model.eval()
print("Train Accuracy:",accuracy(X_train,y_train))
print("Test Accuracy:",accuracy(X_test,y_test))

Train Accuracy: tensor(0.6759, device='cuda:0')
Test Accuracy: tensor(0.6362, device='cuda:0')


_________________________________________________

In [15]:
# DATASET X1

class Net1(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.LazyLinear(256),
        nn.Dropout(p=0.5),
        nn.ReLU(),
        nn.LazyLinear(512),
        nn.Dropout(p=0.5),
        nn.ReLU(),
        nn.LazyLinear(1024),
        nn.Dropout(p=0.5),
        nn.LazyLinear(40)
    )

  def forward(self, x):
    return self.net(x)

In [16]:
import torch.nn.functional as F

# Hyperparameters
batch_size = 500
n_epochs = 2000
lr = 10e-4

# Creating dataloader
torch.manual_seed(0)
train_iter = dataloader(X1_train,y_train,batch_size)

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Assigning to GPU if available

model = Net1().to(device) # Model
loss_fn = nn.CrossEntropyLoss() # Loss function
optimizer = torch.optim.Adam(model.parameters(),lr=lr) # Optimizer

model.apply(init_weights) # Initialize Weights

for epoch in range(1,n_epochs+1):
  total_loss = 0
  for x_batch, y_batch in train_iter:

    # Geting batches
    x_batch = x_batch.to(device)
    y_batch = F.one_hot(y_batch.reshape(-1).long(),40).to(device).float()

    model.train() # Puting model into training mode
    yhat = model(x_batch) # Get prediction
    loss = loss_fn(yhat, y_batch) # Calculate loss

    # Updating weights
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss += loss/batch_size

  if not epoch % 50:
    model.eval()
    print(epoch,total_loss)
    print(accuracy(X1_train,y_train),accuracy(X1_valid,y_valid))

50 tensor(0.6008, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5718, device='cuda:0') tensor(0.5347, device='cuda:0')
100 tensor(0.5882, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5908, device='cuda:0') tensor(0.5444, device='cuda:0')
150 tensor(0.5823, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5956, device='cuda:0') tensor(0.5412, device='cuda:0')
200 tensor(0.5792, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6006, device='cuda:0') tensor(0.5413, device='cuda:0')
250 tensor(0.5759, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6031, device='cuda:0') tensor(0.5391, device='cuda:0')
300 tensor(0.5752, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6110, device='cuda:0') tensor(0.5433, device='cuda:0')
350 tensor(0.5713, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6127, device='cuda:0') tensor(0.5442, device='cuda:0')
400 tensor(0.5711, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6130, device='cuda:0') tensor(0.5402, device='cuda:0')
450 tenso

In [17]:
# Evaluating the model

model.eval()
print("Train Accuracy:",accuracy(X1_train,y_train))
print("Test Accuracy:",accuracy(X1_test,y_test))

Train Accuracy: tensor(0.6288, device='cuda:0')
Test Accuracy: tensor(0.5155, device='cuda:0')


_________________________________________________

### Answer Q.3

To check overfitting and performance of the model, the train and validation accuracy were printed out (every 50 epochs). After selecting parameters based on the best validation, the test accuracy was also taken. While dataset X performed worse in Question 2, it was able to perform better here in Question 3. It was able to reach a higher accuracy without overfitting too much.

On the otherhand, dataset X1 was much more prone to overfitting and at a faster rate, resulting in a lower accuracy. Even though a lower number of epochs were used and dropout layers were used, it still tended to overfit. Simpler models with less layers, however, tended to underfit.

The results for the two datasets are summarized below:

Dataset X (lr=10e-3, batch_size=500,epochs=2000, layers=(1000,500), activation=ReLU, dropout_layers=False):
- Train Accuracy: 67.59%
- Validation Accuracy: 64.06%
- Test Accuracy: 63.62%

Dataset X1 (lr=10e-4, batch_size=500,epochs=100, layers=(256,512,1024),activation=ReLU, dropout_layers=True):
- Train Accuracy: 62.88%
- Validation Accuracy: 53.84%
- Test Accuracy: 51.55%

In [ ]:
# for grader use only

# insert grade here  (out of 8)

# G[3] =
#
# please justify point subtractions when needed

In [ ]:
# total score
max_score = 20
final_score = sum(G)*(100/max_score)